In [1]:
%matplotlib inline

import pymc3 as pm
from pymc3.ode import DifferentialEquation
import numpy as np
import matplotlib.pyplot as plt
import seir_func_point as seir_func
import map_functions
import arviz as az
import theano

plt.style.use('seaborn-darkgrid')

In [2]:
city = "rj"
time_stamp = 3 * 365
incidence_real = map_functions.get_cases(city=city)[1:time_stamp]
infected_percentage = incidence_real[0]/map_functions.pop_dict[city]

In [3]:
seir = seir_func.seir()

In [4]:
seir_model = DifferentialEquation(
    func=seir.derivate,
    times=np.arange(1, time_stamp, 1),
    n_states=10,
    n_theta=16,
    t0=0,
)

In [5]:
with pm.Model() as model4:
    sigma = pm.HalfCauchy('sigma', 1, shape=1, testval = 1)

    # Distribuições a priori
    # R0 é limitada inferiormente em 1 para sempre termos uma epidemia.
    SH = pm.Uniform("SH", 0, 1, testval = 0.4891611167313461,)
    IH = infected_percentage
    EH = infected_percentage
    RH = 1 - SH - 2 * infected_percentage
    
    
    a_t = pm.Uniform("a(t)", 0.5, 2, testval = 0.691410701905391)
    p_mh_t = pm.Uniform("p_mh(t)", 0.5, 2, testval = 1.5609791900930738)
    u_v = pm.Uniform("u_v", 0.5, 2, testval = 1.1218292459714525)
    k = pm.Uniform("k", 0.5, 2, testval = 1.0323746614550986)
    gamma = pm.Uniform("\gamma", 0.5, 2, testval = 0.5232776467454814)
    delta = pm.Uniform("\delta", 0.5, 2, testval = 0.9923266319695812)
    

    sir_curves = seir_model(y0=[
        1000, 0, 0.99, 0, 0.0, 
        SH, IH, EH, 
        RH, IH
    ], 
                           theta=[1, 1, a_t, 
                  1, p_mh_t, 
                  1, 1, u_v, 
                  1, 1, 1, k, 
                  1, gamma, 
                  delta, 1]
                           )

    incidence = pm.Gamma('incidence', 
                     mu=sir_curves[:,9], 
                     sigma=sigma, 
                     observed=incidence_real)
    
    #db = pm.backends.Text('database/traces.csv') # Salva as amostras e assim evita de manter tudo na memória
    
    trace = pm.fit(method="advi", n = 20)
    #trace = pm.sample(2000, tune=1000)#, cores=4)

Finished [100%]: Average Loss = 34,849


In [7]:
pm.save_trace(trace, 'database/linreg.trace') 

AttributeError: 'MeanFieldGroup' object has no attribute '_straces'

In [10]:
#inference    
tr1 = trace.sample(draws=1000)
pm.plots.traceplot(tr1);    
pm.plot_posterior(tr1, color='LightSeaGreen');

plt.plot(approx.hist)

/usr/local/lib/python3.6/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


KeyboardInterrupt: 

In [6]:
data = az.from_pymc3(trace=trace)
data

/usr/local/lib/python3.6/dist-packages/arviz/data/io_pymc3.py:92: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


AttributeError: 'MeanFieldGroup' object has no attribute '_straces'

In [ ]:
az.plot_posterior(data, round_to=2, hdi_prob=0.95)

In [1]:
string = '1jo7h-pauper-matinal-todas-quintas-as-\u200b2503'
string

'1jo7h-pauper-matinal-todas-quintas-as-\u200b2503'